In [ ]:
!wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz

In [ ]:
!tar -xvzf yelp_review_polarity_csv.tgz

In [ ]:
from fastai import *
from fastai.text import *
from fastai.core import *

In [ ]:
path = Path('yelp_review_polarity_csv')

In [ ]:
train_csv = path/'train.csv'
train = pd.read_csv(train_csv, header=None)
train.head()

In [ ]:
train.iloc[0][1]

In [ ]:
valid_csv = path/'test.csv'
valid = pd.read_csv(valid_csv, header=None)
valid.head()

In [ ]:
data_lm = TextLMDataBunch.from_csv(path, valid='test')
data_clas = TextClasDataBunch.from_csv(path, valid='test', vocab=data_lm.train_ds.vocab)

In [ ]:
data_lm.train_ds.vocab_size

In [ ]:
data_lm.train_ds.vocab.itos

In [ ]:
data_lm.train_ds.labels

In [ ]:
data_lm.valid_ds.labels

In [ ]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [ ]:
learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [ ]:
learn.save_encoder('fine_tuned')